In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re #Regular Expression
from nltk import PorterStemmer #Natural language kit used to remove the 'ing' ,'ly','s' etc
'''WordNetLematizer is another method used to remove the ending .The main diff is Stemmer will not knowing the context , 
or actual meaning, but lemantizer will consider the dictonary meaning'''
from wordcloud import WordCloud,ImagColorGenerator  #used to create wordcloud
from PIL import Image #Used to get the image and convert to array
import requests # used for HTTP requests 
import urllib

ModuleNotFoundError: No module named 'wordcloud'

In [ ]:
#NLTK - tokenisation, stemming,tagging,parsing etc.
from nltk.tokenize import word_tokenize #used to tokenize the words
#import nltk
#nltk.download('punkt')

In [ ]:
train_data=pd.read_csv('https://raw.githubusercontent.com/dD2405/Twitter_Sentiment_Analysis/master/train.csv')

In [ ]:
#Generation copy of original file
train_backup=train_data.copy()

In [ ]:
train_data.head()

In [ ]:
test_data=pd.read_csv('https://raw.githubusercontent.com/dD2405/Twitter_Sentiment_Analysis/master/test.csv')
test_backup=test_data.copy()

In [ ]:
test_data.head()


In [ ]:
#Data Pre Processing
#Combining train and test dataset 
#Other option is to add one more column where specify 'train' or 'test'
data=train_data.append(test_data,ignore_index=True,sort=True)
data.tail()

In [ ]:
#Removing the Twitter handles @user, as this not contributing anything for labeling
#Created function to remove unwnted pattern
def remove_pattern(text,pattern):
    pattern_list=re.findall(pattern,text)  #Find all patterns in the text and create a list of them like ['@user', '@user']
    
    for temp in pattern_list:
        text=re.sub(temp,"",text) #substitute each pattern with "" in the text
    return text
    

In [ ]:
#Method 1 -- tweet is first parameter and second parameter given in apply function 
data['modi_tweet']=data['tweet'].apply(remove_pattern,pattern='@[\w]*')
data.head()
#Method 2 --using numpy.vectorize--To improve the speed compared to apply()
#data['modi_tweet']=np.vectorize(remove_pattern)(data['tweet'],'@[\w]*')

In [ ]:
#Remove all special charcaters, numbers ,punctuations etc..
#[^a-zA-Z#] --one charcater not in the list will be replaced by " "
data['modi_tweet']=data['modi_tweet'].str.replace("[^a-zA-Z#]"," ")
data.head()

In [ ]:
data.head(20)

In [ ]:
#Removing words which have max of 3 letters length. or we can say stopwords
#Jion function will join the ' ' with each woord in the loop
#split() function will spit the words based on space 
data['modi_tweet']=data['modi_tweet'].apply(lambda x:' '.join(w for w in x.split() if len(w)>3 ))
data.head()
'''The stop words can be removed from the document by using the stopwords package
from nltk.corpus import stopwords
stopwrds=set(stopwords.words('english'))
words_without_stopwords=[w for w in x.spli() if w not in stopwrds]'''

In [ ]:
ps=PorterStemmer()
tokens=data['modi_tweet'].apply(lambda x:x.split()).apply(lambda x:[ps.stem(i) for i in x])
tokens

In [ ]:
for i in range(len(tokens)):
    tokens[i]=' '.join(tokens[i])
data['modi_tweet']=tokens
data.head()

In [ ]:
#Data Visualisation
#Wordcloud is most commonly used method --- Here most freauent word s will disply in largesize
#Get all label 0 (positive) words
pos_words=' '.join(w for w in data['modi_tweet'][data['label']==0])
pos_words